# Spectral Classification using templates

In (Jianfeng Wu, Jonker et al 2015) they:
    
 > present Gemini spectroscopy for 21 candidate optical counterparts to X-ray sources
discovered in the Galactic Bulge Survey (GBS). 

For these sources they classify the spectra using a set of template stars. The set of template stars was chosen from the **library of the Ultraviolet and Visual Echelle Spectrograph Paranal Observatory Project** ([UVES POP](http://www.eso.org/sci/observing/tools/uvespop.html); Jehin et al. 2005), covering spectral types from A0 to M6 with luminosity class of V. The UVES POP template spectra provide coverage over the wavelength range of 3000–10000 Å with a spectral resolution of ∼ 80,000. 

> The templates were **re-sampled and Gaussian-smoothed** to match the
spectral resolution of the object spectra. The object spectra were
**Doppler-shifted into the same rest frame and averaged**. Each stellar template was optimally subtracted from the object spectrum, while a $\chi^2$ test was performed on the residuals. All the emission lines, DIBs, and telluric features (e.g., Kurucz 2006; Wallace et al. 2011) were masked during the procedure. 


For example from figure 8 from the article:

![](images/template.png)



# This notebook

The purpose of this notebook is to try to recreate what they did to be able to classify spectra for the sources I am working with. 

## A0V HD 162305

In table 4 of (Jianfeng Wu, Jonker et al 2015) they list the templates used. For the spectral type A0V they used **HD 162305** in [NGC 6475](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=HD+162305&NbIdent=1&Radius=10&Radius.unit=arcmin&jsessionid=2D83A5F09A4B709E1A5077131FB78366)

First we import the required packages

In [1]:
#Import packages
from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
from astropy.convolution import convolve, Box1DKernel
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation, CheckboxGroup
from bokeh.palettes import viridis, BrBG8, Accent8, OrRd8, Pastel2_4
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

After downloading the data for the template star from http://www.eso.org/sci/observing/tools/uvespop/interface.html load the data. The data comes in the format of a fits table. We can look at the header to see what each columns represent and how many rows and columns it has. 

In [2]:
t = fits.open('stars/hd162305.tfits')
data = t[1].data
t[1].header[3:20]

NAXIS1  =                   24 / Characters in a row                            
NAXIS2  =               411803 / No. of rows in table                           
PCOUNT  =                    0 / Parameter count always 0                       
GCOUNT  =                    1 / Group count always 1                           
TFIELDS =                    3 / No. of columns in table                        
                                                                                
TFORM1  = '1D      '           / Format of field                                
TDISP1  = 'E15.5   '           / Display format of field                        
TTYPE1  = 'Wave    '           / Field label                                    
TUNIT1  = '        '           / Physical unit of field                         
TFORM2  = '1D      '           / Format of field                                
TDISP2  = 'E15.5   '           / Display format of field                        
TTYPE2  = 'Flux    '        

From the header above we know that the data has three columns: Wave, Flux and Sigma. Also from the [website](http://www.eso.org/sci/observing/tools/uvespop/legend.html#column3). 

> This final file is a tfits table with 3 columns : (1) the wavelengths rebinned to the heliocentric rest frame, (2) the flux of the combined spectra (on an arbitrary scale) and (3) the associated standard deviation.

We get the wave and flux data to plot it. It has 411803 lines to it takes a few seconds and I had to increase the data rate limit of the Jupyter notebook with *--NotebookApp.iopub_data_rate_limit=100000000*. uncomment to run the code and plot the full spectrum. 

In [3]:
waveall = data['Wave']
fluxall = data['Flux']
#source = ColumnDataSource(data=dict(x=waveall,y=fluxall))
#hover = HoverTool(
#        tooltips=[
#            #("index", "$index"),
#            ("(x,y)", "($x{1.11}, $y)"),
#        ]
#    )

#plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra of an A0V: HD 162305",plot_width=900, plot_height=700)
#plot.add_tools(hover)
#plot.add_tools(tools.ResizeTool())
#plot.line('x','y',source=source)
#show(plot)

# Downsampling by decimation?


We see that it takes a while to plot the whole spectra since it has 411803 rows, many more compare to the 2499 row of the data from VIMOS. We can examine the VIMOS data and compare the wavelenght steps and number of elements for both data sets. 

## Sample CX Source: CX59

In each folder we create the spectra for each source. We can recicle the data. For example for CX59 after executing the Notebook for CX59 we select the .txt to plot and compare to the templates. Any changes to the notebook, just execute this cell again to have the updated extracted spectra from the VIMOS source.


In [4]:
folder = '../cx0059/'
notebook = 'CX59.ipynb'
! jupyter nbconvert --ExecutePreprocessor.timeout=600 \
          --ExecutePreprocessor.kernel_name="python2" --to notebook \
          --execute {folder}{notebook}  --output {folder}{notebook}
! ls {folder}

[NbConvertApp] Converting notebook ../cx0059/CX59.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python2
[NbConvertApp] Writing 2291738 bytes to ../cx0059/../cx0059/CX59.ipynb
CX59.ipynb
VI_SEXM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
VI_SRFM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
VI_SSEM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
cx59ssem.dispcor.fits
cx59ssem.fits
cx59ssem.ms.fits
cx59ssem.txt
cx59ssemcont.dispcor.fits
cx59ssemcont.fits
cx59ssemcont.txt
database
images
logfile
login.cl
pyraf
uparm


We select the **.txt** we want to examine the VIMOS data and compare the wavelenght steps and number of elements for both data sets. 

In [5]:
#Sample data
sourcename = folder+'cx59ssem.txt'
xspectra=[]
yspectra=[]
with open(sourcename) as f:
    for lines in f:
        xspectra.append(float(lines.split()[0]))
        yspectra.append(float(lines.split()[1]))

xspectra = np.array(xspectra)
print('VIMOS data steps: min {} max {}'.format(np.diff(xspectra).min(), np.diff(xspectra).max()))
print('Template data steps: min {} max {}'.format(np.diff(waveall).min(), np.diff(waveall).max()))
print('Rows VIMOS data: '+str(len(xspectra)))

VIMOS data steps: min 2.6 max 2.6
Template data steps: min 0.0124886001017 max 0.0516544209804
Rows VIMOS data: 2499


We can notice that the steps for the template are much shorter and also seem to be uneven. One first try to be able to compare both dataset is to only take from the templates those wavelenghts and fluxes closer to the VIMOS data. 

In [6]:
equalwaves = []
equalflux = []
for i in xspectra:
    indx = np.where(abs(waveall-i) < 0.1)
    equalwaves.append(waveall[indx][0])
    equalflux.append(fluxall[indx][0])
print('VIMOS data steps: min {} max {}'.format(np.diff(xspectra).min(), np.diff(xspectra).max()))
print('New Template data steps: min {} max {}'.format(np.diff(equalwaves).min(), np.diff(equalwaves).max()))
print('Rows New VIMOS data: '+str(len(equalwaves)))
print('Maximum separation: {}'.format(abs(equalwaves -xspectra).max()))

VIMOS data steps: min 2.6 max 2.6
New Template data steps: min 2.58272104898 max 2.61970865371
Rows New VIMOS data: 2499
Maximum separation: 0.0999976922049


Now it is more equally distributed around 2. 6. It is still not totally uniform step, but now it has the same number of elements as the VIMOS data. The largest difference in wavelenght between both data set is of 0.099. We can plot it. 

In [7]:
source = ColumnDataSource(data=dict(x=equalwaves,y=equalflux))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra of an A0V: HD 162305",plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

# Gaussian-smoothing

One of the steps mentioned in the paper was to do a gaussian smooth. This can be done in several ways with python. One using scipy package and the routine [scipy.ndimage.filters.gaussian_filter](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.filters.gaussian_filter.html) or [Astropy.convolution](http://docs.astropy.org/en/stable/convolution/).

The Gaussian1Dkernel takes as required argument the standard deviation of the Gaussian kernel. 

We can try to do a smoothing of the recently selected wavelenght data from the template


In [8]:
# Do the smoothing
fluxgaussian10 = convolve(equalflux, Gaussian1DKernel(stddev=2))
#Plot
source = ColumnDataSource(data=dict(x=equalwaves,y=fluxgaussian10))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Smoothed Spectra",plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

## Normalizing the spectrum

We need to normalize both spectra to compare them. 

We use the iraf routine continuum. We can define a sample to correctly normalize the desire range of wavelenght. For the CX source we use the spectra created in [Notebook CX59](../cx0059/CX59.ipynb)

### For the CX source

In each folder we create the spectra for each source. We can recicle the data. For example for CX59 after executing the Notebook for CX59 we select the .txt to plot and compare to the templates. Any changes to the notebook, just execute this cell again to have the updated extracted spectra from the VIMOS source.

In [9]:
folder = '../cx0059/'
notebook = 'CX59.ipynb'
! jupyter nbconvert --ExecutePreprocessor.timeout=600 \
          --ExecutePreprocessor.kernel_name="python2" --to notebook \
          --execute {folder}{notebook}  --output {folder}{notebook}
! ls {folder}

[NbConvertApp] Converting notebook ../cx0059/CX59.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python2
[NbConvertApp] Writing 2291738 bytes to ../cx0059/../cx0059/CX59.ipynb
CX59.ipynb
VI_SEXM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
VI_SRFM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
VI_SSEM_575273_2011-05-26T05:19:50.824_G475_MR_202166_Q2_hi.fits
cx59ssem.dispcor.fits
cx59ssem.fits
cx59ssem.ms.fits
cx59ssem.txt
cx59ssemcont.dispcor.fits
cx59ssemcont.fits
cx59ssemcont.txt
database
images
logfile
login.cl
pyraf
uparm


We select the **.txt** we want to use. 

In [10]:
sourcename = folder+'cx59ssemcont.txt'
#Plotting
xr = (7700,8900)
yr = (0.5,1.4)

xcx=[]
ycx=[]

with open(sourcename) as f:
    for lines in f:
        xcx.append(float(lines.split()[0]))
        ycx.append(float(lines.split()[1]))

#toxcx = np.array(xcx)
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=xcx,y=ycx))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

## For the Smooth and decimated template

From the data need to create a fits image to apply continuum iraf routine to it. One way to do this is show below. There are probably better ways. I am assuming that the step separtion is 2.6 so has the same wavelength axis than the VIMOS data. Lets look at the header of the VIMOS data

In [11]:
cx = fits.open('../cx0059/cx59ssem.ms.fits')
cx[0].header[0:34]

SIMPLE  =                    T / Fits standard                                  
BITPIX  =                  -32 / Bits per pixel                                 
NAXIS   =                    3 / Number of axes                                 
NAXIS1  =                 2499 / Axis length                                    
NAXIS2  =                    1 / Axis length                                    
NAXIS3  =                    4 / Axis length                                    
EXTEND  =                    T / File may contain extensions                    
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
IRAF-TLM= '2017-06-29T22:19:15' / Time of last modification                     
OBJECT  = '453432_GBS'         / Name of the object observed                    
COMMENT FITS (Flexible Image Transport System) format is defined in 'Astronomy  
COMMENT and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H   
DATE    = '2017-06-29T22:19:

The relevant header information are NAXIS#, CRVAl#, CTYPE, CD#_#. We copy this to the header of the new fits woth the smoothed template data we can create. 

In [12]:
name = 'fluxsmooth'
if os.path.exists(name+'.fits'):
    os.remove(name+'.fits')
hdu = fits.PrimaryHDU(fluxgaussian10)
hdu.writeto(name+'.fits')
data, header = fits.getdata(name+'.fits', header=True)
header['NAXIS'] = 2
header['NAXIS2'] = 2499
header['CRVAL1'] = 3501.3
header['CRPIX1'] = 1.
header['CTYPE1'] = 'PIXEL'
header['CTYPE2'] = 'LINEAR'
header['CD1_1'] = 2.6
header['CD2_2'] = 1.
header
fits.writeto(name+'.fits', data, header, overwrite=True)

### Smoothed template Continuum

Now we can normalize using iraf continuum. 

In [13]:
sourcename = name
sample = '*'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

#Plotting
#Plotting
xr = (5000,8900)
yr = (0.5,1.4)

#Not sure if I need to do this
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))

x = np.array(x)
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

fluxsmoothcont.fits: Resampling using current coordinate system
fluxsmoothcont.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9996.1, dw =      2.6, nw = 2499


# Together
 We can plot both graphs after being normalized

In [14]:
xr = (7700,8900)
yr = (0.5,1.4)
p =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
r = p.line(x=x, y=y,color='firebrick',line_width=4,line_alpha=0.8,legend='CX Source')
p.line(x=xcx,y=ycx,color='navy', line_alpha=0.3, line_width=4,legend='Template',muted_alpha=0.1)
p.legend.location = "top_left"
p.legend.click_policy="mute"
show(p)

# Add all V class template stars

For a given VIMOS source we can plot the VIMOS spectra and all spectral types. A total of 28 template spectra. In the future possible include other luminosity class and try to make the selection of the template a widget and not like right now that it is an interactive legend. 

It takes a long time to get the flux and wavelenght values to normalize the 28 spectra so only run the whole code if need to create again the template spectra to plot. To overlay the template star click on the name of the desired template to overlay. The names are as follow:

| Spectral Type | Star name |
|:-------------:|:---------:|
| A0V           | HD 162305 |
| A1V           |  HD 65810 |
| A2V           |  HD 60178 (Castor) |
| A3V           | HD 211998 |
| A4V           | HD 145689 |
| A5V           |  HD 39060 |
| A7V           | HD 187642 (Altair) |
| A9V           |  HD 26612 |
| F0V           | HD 109931 |
| F1V           |  HD 40136 |
| F2V           |  HD 33256 |
| F3V           |  HD 18692 |
| F4V           |  HD 37495 |
| F6V           |  HD 16673 |
| F8V           |  HD 45067 |
| F9V           |  HD 10647 |
| G0V           | HD 105113 |
| G1V           |  HD 20807 |
| G2V           |  HD 14802 |
| G3V           | HD 211415 |
| G4V           |  HD 59967 |
| G5V           |  HD 59468 |
| G6V           | HD 140901 |
| G9V           |  HD 25069 |
| K2V           |  HD 22049 |
| K5V           |  HD 10361 |
| M0V           | HD 156274 |
| M6V           |  HD 34055 |


In [15]:
#Dictionary of sources name and spectral type
dicsources={"hd162305":'A0V',"hd65810":'A1V', "Castor":"A2V",'hd211998':"A3V",
            "hd145689":'A4V',"hd39060":"A5V", "Altairus":'A7V', "hd26612":'A9V',
           "hd109931":'F0V',"hd40136":'F1V',"hd33256":"F2V","hd18692":"F3V",
           "hd37495":"F4V","hd16673":"F6V","hd45067":"F8V","hd10647":"F9V",
           "hd105113":"G0V","hd20807":"G1V","hd14802":"G2V","hd211415":"G3V",
           "hd59967":"G4V","hd59468":"G5V","hd140901":"G6V","hd25069":"G9V",
           "hd22049":"K2V","hd10361":"K5V","hd156274":"M0V","hd34055":"M6V"}

#To ordered the tempaltes by their spectral type I had to do in the command line:
# mv hd187642.tfits ltairus.tfits
# mv hd60178.tfits Castor.tfits


#Sort by spectral type:
dire = './stars' #Directory with all the template spectra
sortedlist = sorted(dicsources.items(), key=lambda t: t[1])
stars = [ i[0] for i in sortedlist]

#Can have one continuos color bar or a different one per spectral type
#colors = viridis(len(stars)) # colors map to plot
colors = BrBG8 + Accent8 + OrRd8 + Pastel2_4


#plot parameters
xr = (7700,8900)
yr = (0.5,1.4)
p =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )
p.line(x=xcx, y=ycx,color='firebrick',line_width=4,line_alpha=0.8,legend='CX Source')

hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
p.add_tools(hover2)
p.add_tools(tools.ResizeTool())


#Loop over the stars to create plot them on Bokeh plot
#This loops take time so only run if the final product is not available
#if want to delete them to rerun the loop change delete to True:
deletealldata = False
if deletealldata:
    if  os.path.exists(sourcename+'cont.dispcor.fits'):
        os.remove(sourcename+'cont.dispcor.fits')
    if  os.path.exists(name+'.fits'):
        os.remove(name+'.fits') 
    if  os.path.exists(sourcename+'cont.fits'):
        os.remove(sourcename+'cont.fits')
        
        
for index, star in enumerate(stars):

    #Get template star table data
    t = fits.open("{}/{}.tfits".format(dire, star))
    #Get name of the star
    name= t[1].header['HIERARCH ESO OBS TARG NAME']
    sourcename = name
   

    if not os.path.exists(sourcename+'cont.txt'):
    
        # Get wavelenght and flux values
        data = t[1].data
        waveall = data['Wave']
        fluxall = data['Flux']
        #Get wavelenght and flux values near the VIMOS data
        equalwaves = []
        equalflux = []
        for i in xspectra:
            indx = np.where(abs(waveall-i) < 0.1)
            equalwaves.append(waveall[indx][0])
            equalflux.append(fluxall[indx][0])
        sourcename = name

        # Do the smoothing
        fluxgaussian10 = convolve(equalflux, Gaussian1DKernel(stddev=2))

            #Create a fits file from the data
        #name = 'fluxsmooth'
        if os.path.exists(name+'.fits'):
            os.remove(name+'.fits')
        hdu = fits.PrimaryHDU(fluxgaussian10)
        hdu.writeto(name+'.fits')
        data, header = fits.getdata(name+'.fits', header=True)
        header['NAXIS'] = 2
        header['NAXIS2'] = 2499
        header['CRVAL1'] = 3501.3
        header['CRPIX1'] = 1.
        header['CTYPE1'] = 'PIXEL'
        header['CTYPE2'] = 'LINEAR'
        header['CD1_1'] = 2.6
        header['CD2_2'] = 1.
        header
        fits.writeto(name+'.fits', data, header, overwrite=True)
        #Normalize the template smoothed and decimated
        sourcename = name
        sample = '*'
        if os.path.exists(sourcename+'cont.fits'):
            os.remove(sourcename+'cont.fits')
        iraf.noao.onedspec.continuum.setParam('input',sourcename+'.fits')
        iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
        iraf.noao.onedspec.continuum.setParam('interactive','no')
        iraf.noao.onedspec.continuum.setParam('sample',sample)
        iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
        iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')


        #Not sure if I need to do this
        if os.path.exists(sourcename+'cont.dispcor.fits'):
            os.remove(sourcename+'cont.dispcor.fits')

        iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
        iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

    x=[]
    y=[]
    with open(sourcename+'cont.txt') as f:
        for lines in f:
            x.append(float(lines.split()[0]))
            y.append(float(lines.split()[1]))
            
    #If you want to have the name and not the spectra type change legend=name
    t = p.line(x=x,y=y,color=colors[index], line_alpha=0.0, 
           line_width=4,legend=dicsources[name],muted_alpha=1.,muted_color=colors[index])
    #This is a very ideficient way to have the colors in the legend to show up by default. 
    t2 = p.line(x=x[0],y=y[0],color=colors[index], line_alpha=1.0, 
           line_width=4,legend=dicsources[name],muted_alpha=1.,muted_color=colors[index])

    
p.legend.location = "top_left"
p.legend.click_policy="mute"
show(p)

# Still to do

- [ ] Check the normalization for the CX sources
- [ ] Check how to do the shift on the wavelenghts
- [ ] Include other luminosities classes. 
- [ ] Mask emission lines, DIBs, and telluric features
- [ ] Correctly do downsampling by decimation
- [ ] Try other Downsampling methods like by interpolation
- [ ] Check the right value for the Gaussian Smoothing
- [ ] Do the $\chi ^2$ with the residuals
- [ ] Better way to display legends

# References

Jianfeng Wu, P. G. Jonker, M. A. P. Torres, C. T. Britt, C. B. Johnson, R. I. Hynes, S. Greiss, D. T. H. Steeghs, T. J. Maccarone, C. O. Heinke, T. Wevers; Gemini spectroscopy of Galactic Bulge Sources: a population of hidden accreting binaries revealed?. Mon Not R Astron Soc 2015; 448 (2): 1900-1915. doi: https://doi.org/10.1093/mnras/stv047

Jehin, E., Bagnulo, S., Melo, C., Ledoux, C., & Cabanac, R. (2005). The UVES Paranal Observatory Project: a public library of high resolution stellar spectra. Proceedings of the International Astronomical Union, 1(S228), 261–262. http://doi.org/10.1017/S1743921305005739

Kurucz R. L., 2006, preprint (arXiv:https://arxiv.org/abs/astro-ph/0605029)

Wallace L., Hinkle K. H., Livingston W. C., Davis S. P., 2011, ApJS, 195, 6 http://dx.doi.org/10.1088/0067-0049/195/1/6